特徴量の重みを正常に反映しているか確認するため、オッズ系統を全て抜いて学習させる。

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost as cb
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
data_dir = "F:/TARGET frontier/TXT34/"
data_name = "train"
tail = "fe2_1"
train_data = pd.read_csv(data_dir+data_name+tail+".csv",encoding="shift-jis",dtype={'レースID(旧)':str})
#test_data
test_data = pd.read_csv(data_dir+"predict"+tail+".csv",encoding="shift-jis",dtype={'レースID(旧)':str})

In [3]:
pd.set_option('display.max_columns', 100)
train_data.head()

,レースID(旧),場所,芝・ダ,距離,クラス名,馬場状態,頭数,騎手,枠番,性別,年齢,キャリア,斤量,間隔,休み明け〜戦目,種牡馬,父タイプ名,所属,調教師,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,前走レースID(旧),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走PCI3,前走RPCI,前PCI,前好走,前走確定着順,前走着差タイム,前走単勝オッズ,場所距離芝ダ,前走場所距離芝ダ,前走レースID(旧)1,前走場所1,...,前走着差タイム3,前走単勝オッズ3,場所距離芝ダ3,前走場所距離芝ダ3,前走レースID(旧)4,前走場所4,前芝・ダ4,前距離4,前クラス名4,前走馬場状態4,前走出走頭数4,替4,前騎手4,前走枠番4,前走脚質4,前走斤量4,前走Ave-3F4,前走上り3F4,前走PCI34,前走RPCI4,前PCI4,前好走4,前走確定着順4,前走着差タイム4,前走単勝オッズ4,場所距離芝ダ4,前走場所距離芝ダ4,前走レースID(旧)5,前走場所5,前芝・ダ5,前距離5,前クラス名5,前走馬場状態5,前走出走頭数5,替5,前騎手5,前走枠番5,前走脚質5,前走斤量5,前走Ave-3F5,前走上り3F5,前走PCI35,前走RPCI5,前PCI5,前好走5,前走確定着順5,前走着差タイム5,前走単勝オッズ5,場所距離芝ダ5,前走場所距離芝ダ5
0,0615581112,2,2,1200,5,3,16,491,6,2,3,13,56.0,6.0,5.0,669,8,0,363,1,0.0,5.4,1.9,2.7,190.0,0515531003,14,2,1400.0,5,2,15.0,0,330,2.0,3,54.0,37.2,33.9,59.53,58.2,59.7,0,8.0,0.2,14.6,49,49,0815411013,3.0,...,0.9,10.3,24.0,40.0,0515220605,14.0,2.0,1600.0,4.0,3.0,9.0,0.0,330.0,5.0,3.0,56.0,36.90,34.2,59.07,57.7,57.9,0.0,6.0,0.4,6.3,25.0,50.0,0915250603,26.0,2.0,1600.0,4.0,4.0,12.0,0.0,245.0,3.0,4.0,56.0,36.66,35.8,53.87,47.9,52.4,0.0,7.0,1.1,22.6,33.0,149.0
1,0615581111,2,2,1200,5,3,16,77,6,1,3,10,54.0,3.0,3.0,530,8,0,362,2,0.0,2.8,1.3,1.6,130.0,0715420613,1,2,1200.0,4,3,18.0,1,77,7.0,3,54.0,34.5,34.6,48.80,46.9,49.7,0,1.0,0.0,2.2,49,14,0315350913,19.0,...,0.0,7.6,24.0,39.0,0215260804,5.0,2.0,1200.0,4.0,2.0,16.0,1.0,349.0,2.0,3.0,52.0,34.80,36.0,47.07,45.0,46.7,0.0,2.0,0.0,4.2,24.0,215.0,0915310701,26.0,2.0,1400.0,10.0,3.0,18.0,0.0,77.0,1.0,3.0,54.0,35.18,35.3,50.30,48.2,49.6,0.0,1.0,0.0,2.0,77.0,148.0
2,0615581106,2,2,1200,5,3,16,245,3,2,3,9,56.0,5.0,2.0,707,5,1,241,3,0.2,6.2,1.8,2.5,180.0,0815551217,3,2,1200.0,5,3,18.0,0,159,8.0,2,56.0,34.5,34.4,50.13,48.0,50.3,0,7.0,0.4,28.6,49,177,0815381006,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0615581108,2,2,1200,5,3,16,390,4,1,5,17,55.0,2.0,4.0,106,1,1,377,4,0.3,6.6,1.7,2.5,0.0,0615540911,2,2,1200.0,5,2,16.0,1,390,6.0,2,53.0,34.1,34.4,48.17,46.3,49.1,0,3.0,0.0,19.1,49,106,0815460915,3.0,...,0.0,4.6,49.0,85.0,0415141210,12.0,2.0,1000.0,4.0,3.0,16.0,0.0,101.0,5.0,4.0,55.0,34.95,32.8,52.50,49.9,56.6,0.0,4.0,0.3,3.7,43.0,22.0,0315121201,19.0,2.0,1200.0,4.0,3.0,16.0,1.0,101.0,1.0,3.0,55.0,33.90,35.6,46.60,44.1,45.2,0.0,2.0,0.2,5.7,13.0,85.0
4,0615581116,2,2,1200,5,3,16,443,8,1,3,15,54.0,2.0,4.0,390,4,1,186,5,0.3,33.9,6.3,9.7,0.0,0615540904,2,2,1200.0,5,2,16.0,0,249,2.0,5,53.0,33.6,36.4,48.17,46.3,42.3,1,13.0,1.5,8.6,49,106,0315311107,19.0,...,0.0,12.4,49.0,215.0,0215140214,5.0,2.0,1200.0,10.0,4.0,16.0,0.0,287.0,7.0,2.0,54.0,34.80,36.4,44.53,43.2,45.6,0.0,5.0,0.7,16.2,77.0,215.0,0415170515,12.0,2.0,1200.0,10.0,3.0,16.0,0.0,101.0,8.0,4.0,54.0,34.80,35.0,48.67,46.9,49.4,0.0,9.0,0.5,14.6,77.0,23.0


In [4]:
columns_name = ["レースID(旧)","場所","芝・ダ","距離","クラス名","馬場状態","頭数","騎手","枠番","性別","年齢","キャリア","斤量","間隔","休み明け〜戦目","種牡馬","父タイプ名","所属","調教師","確定着順","着差","単勝オッズ","複勝オッズ下限",
                "複勝オッズ上限","複勝配当","前走レースID(旧)","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","替","前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F","前走上り3F","前走PCI3",
                "前走RPCI","前PCI","前好走","前走確定着順","前走着差タイム","前走単勝オッズ","場所距離芝ダ","前走場所距離芝ダ","前走レースID(旧)1","前走場所1","前芝・ダ1","前距離1","前クラス名1","前走馬場状態1","前走出走頭数1",
                "替1","前騎手1","前走枠番1","前走脚質1","前走斤量1","前走Ave-3F1","前走上り3F1","前走PCI31","前走RPCI1","前PCI1","前好走1","前走確定着順1","前走着差タイム1","前走単勝オッズ1","場所距離芝ダ1","前走場所距離芝ダ1",
                "前走レースID(旧)2","前走場所2","前芝・ダ2","前距離2","前クラス名2","前走馬場状態2","前走出走頭数2","替2","前騎手2","前走枠番2","前走脚質2","前走斤量2","前走Ave-3F2","前走上り3F2","前走PCI32","前走RPCI2",
                "前PCI2","前好走2","前走確定着順2","前走着差タイム2","前走単勝オッズ2","場所距離芝ダ2","前走場所距離芝ダ2","前走レースID(旧)3","前走場所3","前芝・ダ3","前距離3","前クラス名3","前走馬場状態3","前走出走頭数3",
                "替3","前騎手3","前走枠番3","前走脚質3","前走斤量3","前走Ave-3F3","前走上り3F3","前走PCI33","前走RPCI3","前PCI3","前好走3","前走確定着順3","前走着差タイム3","前走単勝オッズ3","場所距離芝ダ3","前走場所距離芝ダ3",
                "前走レースID(旧)4","前走場所4","前芝・ダ4","前距離4","前クラス名4","前走馬場状態4","前走出走頭数4","替4","前騎手4","前走枠番4","前走脚質4","前走斤量4","前走Ave-3F4","前走上り3F4","前走PCI34","前走RPCI4",
                "前PCI4","前好走4","前走確定着順4","前走着差タイム4","前走単勝オッズ4","場所距離芝ダ4","前走場所距離芝ダ4","前走レースID(旧)5","前走場所5","前芝・ダ5","前距離5","前クラス名5","前走馬場状態5","前走出走頭数5","替5",
                "前騎手5","前走枠番5","前走脚質5","前走斤量5","前走Ave-3F5","前走上り3F5","前走PCI35","前走RPCI5","前PCI5","前好走5","前走確定着順5","前走着差タイム5","前走単勝オッズ5","場所距離芝ダ5","前走場所距離芝ダ5",
               ]

In [5]:
use_train_columns = ["クラス名","馬場状態","頭数","騎手","枠番","性別","年齢","キャリア","斤量","間隔","休み明け〜戦目","種牡馬","父タイプ名","調教師","単勝オッズ",
                "前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F",
                "前走上り3F","前PCI","前好走","前走着差タイム","前走単勝オッズ","場所距離芝ダ","前走場所距離芝ダ",
                "前クラス名1","前走出走頭数1","前騎手1","前走着差タイム1","前走単勝オッズ1","前走場所距離芝ダ1",
                "前クラス名2","前走出走頭数2","前騎手2","前走着差タイム2","前走単勝オッズ2","前走場所距離芝ダ2"
               ]
cat_feature = ["クラス名","馬場状態","騎手","性別","種牡馬","父タイプ名","調教師","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態",
               "前騎手","前走脚質","前好走","場所距離芝ダ","前走場所距離芝ダ","前クラス名1","前騎手1","前走場所距離芝ダ1","前クラス名2","前騎手2","前走場所距離芝ダ2",
              ]

In [6]:
train_x = train_data[use_train_columns]
test_x = test_data[use_train_columns]

In [7]:
test_y = test_data["着差"]
train_y = train_data["着差"]

In [8]:
#catboostのparams(ベースライン)
params = {
    'depth' : 6,                  # 木の深さ
    'learning_rate' : 0.1,       # 学習率
    'early_stopping_rounds' : 50,
    'use_best_model' : True,
    'random_seed' :42
}

In [9]:
#カテゴリカル変数になっている特徴量の列名を指定
categorical_features_indices = cat_feature

In [10]:
#これがないとエラー履くので注意
train_y.fillna(2.0,inplace=True) #無ければ10位に
test_y.fillna(2.0,inplace=True)

In [11]:
#上記のものは全て文字列またはint型である必要があり、floatや欠損値は変換しなければならない
for i in categorical_features_indices:
    train_x[i].fillna(9999,inplace=True)
    test_x[i].fillna(9999,inplace=True)
for i in categorical_features_indices:
    if train_x[i].dtype != 'int64':
        train_x[i] = train_x[i].astype('int64')
    if test_x[i].dtype != 'int64':
        test_x[i] = test_x[i].astype('int64')

In [12]:
for i in categorical_features_indices:
    print(train_x[i].dtype)

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64


In [13]:
scores = []
histories = []
models = []
y = pd.Series()
kf = KFold(n_splits=4,shuffle=False)
for tr_idx,va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    #watchlist = [(dtrain,"train"),(dvalid,"eval")]
    evals_result = {}
    model = cb.CatBoostRegressor(iterations=10000,
                                 learning_rate=0.01,
                                 depth=6,
                                 verbose=100,
                                 use_best_model=True,
                                 early_stopping_rounds=100,
                                 one_hot_max_size=100,
                                 task_type="GPU",
                                 devices='0:1'
                                 
                                )
    
    model.fit(tr_x,
              tr_y,
              cat_features=categorical_features_indices,
              eval_set=(va_x, va_y),
              plot=False,

             )
    models.append(model)
    #histories.append(evals_result)
    va_pred = model.predict(va_x)
    va_y.fillna(0.5,inplace=True)
    score = mean_squared_error(va_y,va_pred)
    score = np.sqrt(mean_squared_error(va_y,va_pred)) #rmse
    scores.append(score)
    
    #evaluationさせる
    #全部のfoldについてyに入れておいて、
    y_pred_proba = model.predict(va_x)
    y = pd.concat([y,pd.Series(y_pred_proba)])


output = train_data
output = output.assign(ya=y.values)
output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/kfeval2.csv",index=False,encoding="shift-jis")
    
print(*scores)
print(np.mean(scores))

0:	learn: 1.3031753	test: 1.3332528	best: 1.3332528 (0)	total: 42.1ms	remaining: 7m 1s
100:	learn: 1.1456609	test: 1.1967994	best: 1.1967994 (100)	total: 3.91s	remaining: 6m 22s
200:	learn: 1.1097223	test: 1.1690479	best: 1.1690479 (200)	total: 7.71s	remaining: 6m 15s
300:	learn: 1.0962063	test: 1.1600277	best: 1.1600277 (300)	total: 11.7s	remaining: 6m 17s
400:	learn: 1.0873800	test: 1.1548574	best: 1.1548574 (400)	total: 15.7s	remaining: 6m 15s
500:	learn: 1.0805324	test: 1.1511695	best: 1.1511695 (500)	total: 19.8s	remaining: 6m 15s
600:	learn: 1.0744446	test: 1.1484151	best: 1.1484151 (600)	total: 23.8s	remaining: 6m 12s
700:	learn: 1.0689335	test: 1.1465860	best: 1.1465860 (700)	total: 27.8s	remaining: 6m 8s
800:	learn: 1.0646280	test: 1.1451375	best: 1.1451375 (800)	total: 31.8s	remaining: 6m 5s
900:	learn: 1.0603985	test: 1.1442263	best: 1.1442263 (900)	total: 35.9s	remaining: 6m 2s
1000:	learn: 1.0564794	test: 1.1436753	best: 1.1436611 (990)	total: 39.8s	remaining: 5m 58s
1100:

In [ ]:
#acc:3.4866(lr=0.01,depth=7)
#acc:3.4876 (le=0.005)
#acc:3.4871 (le=0.01,depth=7→8)
#acc:3.4869 (lr=0.01,depth=6)
#学習結果のプロット
%matplotlib inline
def plot_loss(evals_result, title):
    train_metric = evals_result['train']['rmse']
    plt.plot(train_metric, label='train rmse')
    eval_metric = evals_result['eval']['rmse']
    plt.plot(eval_metric, label='eval rmse')
    plt.grid()
    plt.legend()
    plt.title(title)
    plt.xlabel('rounds')
    plt.ylabel('rmse')
    plt.show()

In [ ]:
for i in range(4):
    plot_loss(histories[i],f'Training Dataset: {i}')

In [14]:
test_length = len(test_data)
predict = np.array([0.0]*test_length)
for i in range(4):
    y_pred_proba = models[i].predict(test_x)
    predict += y_pred_proba

predict /= 4

In [15]:
y = pd.DataFrame(predict,columns=["y'"])
output = test_data #ここも書き換えること

output = pd.concat([output,y],axis=1)

output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/neweval6.csv",index=False,encoding="shift-jis")

In [16]:
fscore = model.get_feature_importance()

In [17]:
jscore = []
for i,j in zip(use_train_columns,fscore):
    jscore.append((i,np.round(j,decimals=2)))
    

In [18]:
from operator import itemgetter
jscore.sort(reverse=True,key=itemgetter(1))

In [19]:
jscore

[('単勝オッズ', 31.16),
 ('場所距離芝ダ', 15.1),
 ('キャリア', 9.56),
 ('前走着差タイム', 9.03),
 ('クラス名', 4.27),
 ('頭数', 3.9),
 ('前芝・ダ', 3.82),
 ('前好走', 3.12),
 ('馬場状態', 2.2),
 ('前距離', 1.87),
 ('種牡馬', 1.8),
 ('前走上り3F', 1.63),
 ('調教師', 1.5),
 ('前走場所距離芝ダ', 1.39),
 ('前走脚質', 1.2),
 ('間隔', 1.16),
 ('前走着差タイム1', 1.16),
 ('騎手', 1.06),
 ('前PCI', 1.03),
 ('前走単勝オッズ', 0.52),
 ('年齢', 0.42),
 ('前走Ave-3F', 0.33),
 ('前走場所距離芝ダ1', 0.32),
 ('前走場所距離芝ダ2', 0.31),
 ('前クラス名', 0.28),
 ('前騎手', 0.26),
 ('前走斤量', 0.25),
 ('休み明け〜戦目', 0.22),
 ('前クラス名1', 0.16),
 ('前走着差タイム2', 0.15),
 ('枠番', 0.1),
 ('前騎手2', 0.1),
 ('前騎手1', 0.09),
 ('前走出走頭数2', 0.08),
 ('性別', 0.07),
 ('前走馬場状態', 0.07),
 ('前走出走頭数', 0.07),
 ('前クラス名2', 0.07),
 ('前走場所', 0.06),
 ('前走枠番', 0.05),
 ('斤量', 0.02),
 ('前走出走頭数1', 0.01),
 ('前走単勝オッズ1', 0.01),
 ('父タイプ名', 0.0),
 ('前走単勝オッズ2', 0.0)]

In [38]:
#全体を100[%]とした時の重要度
mother = sum([j for i,j in fscore])
for i,j in fscore[:25]:
    print(i,round((j/mother)*100,1))

TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
slen = 20
label = []
weight = []
for i,j in fscore[:slen]:
    label.append(i)
    weight.append(j)

plt.rcdefaults() #横棒グラフモード
fig, ax = plt.subplots()
y_pos = np.arange(slen)
ax.barh(y_pos, weight,align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(label)
ax.invert_yaxis()
ax.set_xlabel('label')
ax.set_title('xgboost特徴量')
plt.show()
    

In [ ]:
#save models
import pickle
with open('xgmachina_ver0.0.7.pickle', mode='wb') as fp:
    pickle.dump(models, fp)

In [ ]:
#import models
"""import pickle
with open('xgmachina_ver0.0.6.pickle', mode='rb') as fp:
    models = pickle.load(fp)"""